In [ ]:
!pip install ray==2.40.0

In [ ]:
!pip install ultralytics

In [8]:
import yaml
import os
from ultralytics import YOLO
from sklearn.model_selection import KFold
import shutil

In [4]:
# turn off wandb!
!wandb disabled

In [ ]:
train_images_path = '/kaggle/input/skeletaldataset/skeletal_dataset/train/images'
train_labels_path = train_images_path.replace('images', 'labels')

In [6]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
image_files = [f for f in os.listdir(train_images_path) if f.endswith('.jpg')]

In [ ]:
model = YOLO('yolov8l-pose.pt')

In [ ]:
fold = 1
for train_idx, val_idx in kfold.split(image_files):
    print(f"Starting Training Fold {fold}...")
    
    fold_dir = f'/kaggle/working/fold{fold}'
    train_img_dest = os.path.join(fold_dir, 'train/images')
    val_img_dest = os.path.join(fold_dir, 'val/images')
    train_lbl_dest = os.path.join(fold_dir, 'train/labels')
    val_lbl_dest = os.path.join(fold_dir, 'val/labels')
    
    os.makedirs(train_img_dest, exist_ok=True)
    os.makedirs(val_img_dest, exist_ok=True)
    os.makedirs(train_lbl_dest, exist_ok=True)
    os.makedirs(val_lbl_dest, exist_ok=True)

    for idx in train_idx:
        fname = image_files[idx]
        shutil.copy(os.path.join(train_images_path, fname), train_img_dest)
        shutil.copy(os.path.join(train_labels_path, fname.replace('.jpg', '.txt')), train_lbl_dest)

    for idx in val_idx:
        fname = image_files[idx]
        shutil.copy(os.path.join(train_images_path, fname), val_img_dest)
        shutil.copy(os.path.join(train_labels_path, fname.replace('.jpg', '.txt')), val_lbl_dest)

    fold_yaml_data = {
        'train': train_img_dest,
        'val': val_img_dest,
        'kpt_shape': [4, 3],
        'names': {0: 'S', 1: 'N', 2: 'A', 3: 'B'}
    }
    
    yaml_file_path = os.path.join(fold_dir, 'data.yaml')
    with open(yaml_file_path, 'w') as f:
        yaml.dump(fold_yaml_data, f)

    model.train(
        data=yaml_file_path,
        epochs=300,
        imgsz=1024,
        batch=16,
        optimizer="AdamW",
        patience=15,
        lr0=0.001,
        lrf=0.1,
        dropout=0.2,
        cos_lr=True,
        pose=15.0,
        kobj=3.0,
        box=15.0,
        cls=1.0,
        weight_decay=0.0005,
        momentum=0.85,
        warmup_epochs=2,
        conf=0.45,
        iou=0.7,
        mosaic=0.0
    )


    shutil.rmtree(os.path.join(fold_dir, 'train'))
    shutil.rmtree(os.path.join(fold_dir, 'val'))
    
    fold += 1

print("\n All 5 Folds completed successfully!")

In [ ]:
# Fold 2 achieved the highest performance 
best_model = YOLO('/kaggle/working/runs/pose/train2/weights/best.pt')
yaml_path = '/kaggle/input/skeletaldataset/skeletal_dataset/dataset.yaml'

metrics = best_model.val(
    data= yaml_path,
    split='test'
)

print(f"Mean Average Precision (mAP50-95): {metrics.box.map}")
print(f"mAP50: {metrics.box.map50}")

In [ ]:

shutil.make_archive("/kaggle/working/final_results2", "zip", "/kaggle/working/runs/pose/train2")

FileLink("/kaggle/working/final_results2.zip")